# Personal Resume Builder

In [1]:
#import needed libraries
from openai import OpenAI
from dotenv import load_dotenv
import json, os, requests
from pypdf import PdfReader
import gradio as gr

In [2]:
#load envirnoment variables
load_dotenv(override=True)
openai = OpenAI()

In [5]:
#lets configure pushover tokens and api link
pushover_user=os.getenv("PUSHOVER_USER")
pushover_token=os.getenv("PUSHOVER_TOKEN")
pushover_url="https://api.pushover.net/1/messages.json"

In [6]:
def push(message):
    print (f'Push: {message}')
    payload = {"user":pushover_user, "token":pushover_token, "message":message}
    requests.post(pushover_url, data=payload)

In [7]:
push("Hi Samson, AI maestro!")

Push: Hi Samson, AI maestro!


In [10]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f'Recording interest from {name} with email {email} and notes {notes}')
    return {"recorded":"ok"}

In [11]:
def record_unknown_question(question):
    push(f'Recording {question} asked that I could not answer')
    return {"recorded":"ok"}

In [13]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [28]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [29]:
tools=[{"type":"function", "function":record_user_details_json},
    {"type":"function", "function":record_unknown_question_json},]

In [30]:
#function to handle tool calls
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print (f'Tool call: {tool_name} with arguments: {arguments}')
        tool = globals()[tool_name]
        result = tool(**arguments)
        results.append({"role":"tool","content":json.dumps(result), "tool_call_id":tool_call.id})
    return results


In [31]:
reader = PdfReader("me/linkedin.pdf")
linkedin_text = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin_text += text
    
with open("me/summary.txt", "r", encoding="utf-8") as f:
    summary=f.read()

name = "Samson Sahadevan"

In [32]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin_text}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [33]:
def chat(message, history):
    messages = [{"role":"system", "content":system_prompt}]+history+[{"role":"user", "content":message}]
    done = False

    while not done:
        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)
        finish_reason = response.choices[0].finish_reason
        print (f'Finish reason: {finish_reason}')
        #if llm wants to make a tool call, handle it
        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done=True
    return response.choices[0].message.content

In [34]:
gr.ChatInterface(chat, type="messages").launch()
                 







* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


Finish reason: tool_calls
Tool call: record_unknown_question with arguments: {'question': 'where did you study 2nd grade'}
Push: Recording where did you study 2nd grade asked that I could not answer
Finish reason: stop
Finish reason: tool_calls
Tool call: record_unknown_question with arguments: {'question': 'what did the teach do samson in 1st grade'}
Push: Recording what did the teach do samson in 1st grade asked that I could not answer
Finish reason: stop
Finish reason: stop
Finish reason: tool_calls
Tool call: record_user_details with arguments: {'email': 'devsaha', 'name': 'testname', 'notes': 'photoshoot'}
Push: Recording interest from testname with email devsaha and notes photoshoot
Finish reason: stop
Finish reason: tool_calls
Tool call: record_unknown_question with arguments: {'question': 'what is dannys name'}
Push: Recording what is dannys name asked that I could not answer
Finish reason: stop
Finish reason: stop
Finish reason: stop
Finish reason: tool_calls
Tool call: record